In [1]:
import cv2
import mediapipe as mp
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

volMin, volMax = volume.GetVolumeRange()[:2]

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    lmList = []
    if results.multi_hand_landmarks:
        for handlandmark in results.multi_hand_landmarks:
            for id, lm in enumerate(handlandmark.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
        mpDraw.draw_landmarks(img, handlandmark, mpHands.HAND_CONNECTIONS)
    if lmList != []:
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cv2.circle(img, (x1, y1), 15, (255, 0, 0), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 0), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
        length = hypot(x2 - x1, y2 - y1)
        vol = np.interp(length, [15, 220], [volMin, volMax])
        print(f"Vol: {vol}, Length: {length}")
        cv2.imshow('Image', img)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break


Vol: -38.95211142490762, Length: 97.62171889492625
Vol: -38.6358253476495, Length: 98.6154146165801
Vol: -38.155360201434725, Length: 100.12492197250393
Vol: -38.066473402531564, Length: 100.40418317978589
Vol: -38.51180713634605, Length: 99.00505037623081
Vol: -38.50698513487319, Length: 99.02019995940222
Vol: -38.82521222425383, Length: 98.02040603874276
Vol: -38.11723553816561, Length: 100.24470060806208
Vol: -38.16966871381106, Length: 100.07996802557443
Vol: -38.180802001751815, Length: 100.04498987955368
Vol: -46.66508996376493, Length: 73.38937252763509
Vol: -46.669427406988675, Length: 73.37574531137656
Vol: -41.81101588241579, Length: 88.63972021616495
Vol: -42.347564844518516, Length: 86.95401083331349
Vol: -42.45759771615816, Length: 86.6083136886985
Vol: -38.59711782305121, Length: 98.73702446397704
Vol: -38.860951986906144, Length: 97.9081201943945
Vol: -36.8577428536751, Length: 104.20172743289815
Vol: -37.11071055252049, Length: 103.40696301506973
Vol: -36.79365855090666

KeyboardInterrupt: 